In [1]:
import ee


In [2]:
ee.Authenticate()


True

In [3]:
ee.Initialize()

In [4]:
aoi=ee.Geometry.Polygon(
[[39.26196452651823,21.893376873910263],
[39.26283624444807,21.893376873910263],
[39.26283624444807,21.893989109128505],
[39.26196452651823,21.893989109128505],
[39.26196452651823,21.893376873910263]]
)#UJ

In [4]:


aoi=ee.Geometry.Polygon([
[39.250070040667005,22.88853586010731],
[39.264017527544446,22.88853586010731],
[39.264017527544446,22.91846156016509],
[39.250070040667005,22.91846156016509],
[39.250070040667005,22.88853586010731]
])

#alnoabih

In [17]:
aoi=ee.Geometry.Polygon([
 [39.20888129773265,21.8537119026787],
 [39.280979076052965,21.8537119026787],
 [39.280979076052965,21.915198230339893],
 [39.20888129773265,21.915198230339893],
 [39.20888129773265,21.8537119026787]
])

In [ ]:
aoi=ee.Geometry.Polygon([

    [46.60554880056072,24.532105031759635],
    [46.751460970970875,24.455873255767536],
    [46.95333475026775,24.496181878263375],
    [46.942005099388844,24.673822919705184],
    [46.908024931598696,24.723718460355094],
    [46.90217965993572,24.783590439033855],
    [46.917210283767645,24.820537820313316],
    [46.88142308758074,24.87555671889163],
    [46.857204379174,24.960354967272554],
    [46.79987949209019,25.008127706608153],
    [46.74013132009197,25.00967921679065],
    [46.58323282155681,25.046076213753985],
    [46.475370394624484,25.00160657236566],
    [46.373805941674,24.90727431598331],
    [46.40471149070172,24.803726426695388],
    [46.49396890554119,24.644358488808862],
    [46.51286518629248,24.544456555430976],
    [46.586666049095875,24.53915317377673],
    [46.60554880056072,24.532105031759635]

]
)

In [4]:
aoi=ee.Geometry.Polygon( [
[46.41388398001249,24.474133938125608],
[46.99272614309843,24.474133938125608],
[46.99272614309843,25.04372308127158],
[46.41388398001249,25.04372308127158],
[46.41388398001249,24.474133938125608]
])  #new RUH-for ndvi values

In [15]:
aoi=ee.Geometry.Polygon( [
[-120.12635677842464,52.14102379672617],
[-120.11297255544038,52.14102379672617],
[-120.11297255544038,52.14779544206277],
[-120.12635677842464,52.14779544206277],
[-120.12635677842464,52.14102379672617]])
#random

In [4]:
aoi=ee.Geometry.Polygon( [
[39.230642953025026,21.578023959235942],
[39.237681069480104,21.578023959235942],
[39.237681069480104,21.583052246512302],
[39.230642953025026,21.583052246512302],
[39.230642953025026,21.578023959235942]
]) #alsamer-2

In [1]:
aoi=ee.Geometry.Polygon( [
[39.23457871724432,21.57825816603353],
[39.234974343074015,21.57825816603353],
[39.234974343074015,21.578487634954122],
[39.23457871724432,21.578487634954122],
[39.23457871724432,21.57825816603353]
])
#alsamer-3

NameError: name 'ee' is not defined

In [20]:
aoi=ee.Geometry.Polygon( [
[46.64972296431122,24.76506189375294],
[46.65386429503022,24.76506189375294],
[46.65386429503022,24.768111145547216],
[46.64972296431122,24.768111145547216],
[46.64972296431122,24.76506189375294]
])
# one hei in ruh

In [4]:
aoi=ee.Geometry.Polygon( [
[46.54538397649305,24.73211203577747],
[46.57988791326063,24.73211203577747],
[46.57988791326063,24.755730996319848],
[46.54538397649305,24.755730996319848],
[46.54538397649305,24.73211203577747]
])
# riadh_Forest 

In [6]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-3-01', '2024-3-28') \
    .filterBounds(aoi) \
    .median()

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'riadh_Forest',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()


# test


In [5]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-3-01', '2024-3-10') \
    .filterBounds(aoi) 
 

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'one_uj',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()


AttributeError: 'ImageCollection' object has no attribute 'clip'

In [15]:
def maskClouds(image):
    # Example of a simple cloud mask using the QA60 band
    # Actual implementations can be more complex and might use additional bands or indices
    qa60 = image.select('QA60')
    cloudMask = qa60.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloudMask)

In [14]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2023-1-01', '2022-12-10') \
    .filterBounds(aoi) \
    .map(maskClouds) \
    .select(['B4', 'B8']) \
    .median()

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'alsamer_new_7',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()


In [18]:
# Filter the Sentinel-2 ImageCollection for the specified date range, AOI, and apply cloud masking
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-01-01', '2024-03-17') \
    .filterBounds(aoi) \
    .map(maskClouds) \
    .select(['B4', 'B8']) \
    .median()  # Compute the median of the filtered collection

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast selected bands to Float32
def castImage(image):
    return image.toFloat()

# Cast the selected bands of the clipped image to Float32
castedImage = castImage(clippedImage)

# Example export parameters (make sure to replace 'aoi.getInfo()['coordinates']' with your AOI coordinates if necessary)
task = ee.batch.Export.image.toDrive(**{
    'image': castedImage,
    'description': 'alsamer_new_11',
    'scale': 10,
    'region': aoi.getInfo()['coordinates'],  # Make sure this is correctly defined for your AOI
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})
task.start()

In [11]:
import datetime
# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Function to export images
def exportImage(image, date):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'RUH-B4-B8_{date.strftime("%Y-%m-%d")}',
        scale=10,
        region=aoi.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        maxPixels=1e9
    )
    task.start()

# Start and end dates
start_date = datetime.datetime(2018, 12, 15)
end_date = datetime.datetime(2024, 1, 30)

# Generate images every 30 days
current_date = start_date
while current_date <= end_date:
    # Define the date range for the current image
    date_range_start = current_date.strftime('%Y-%m-%d')
    next_date = current_date + datetime.timedelta(days=30)
    date_range_end = next_date.strftime('%Y-%m-%d')

    # Get the image collection for the current date range
    sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterDate(date_range_start, date_range_end) \
        .filterBounds(aoi) \
        .map(maskClouds) \
        .select(['B4', 'B8']) \
        .median() 
        
    

    # Clip the image to the AOI
    clippedImage = sentinelCollection.clip(aoi)

    # Cast all bands of the clipped image to Float32
    castedImage = castImage(clippedImage)

    # Export the casted image
    exportImage(castedImage, current_date)

    # Update the current date
    current_date = next_date

print("Images export tasks started.")

Images export tasks started.


# all bands

In [6]:
import datetime
# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Function to export images
def exportImage(image, date):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'RUH-All-Bands_{date.strftime("%Y-%m-%d")}',
        scale=10,
        region=aoi.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        maxPixels=1e9
    )
    task.start()

# Start and end dates
start_date = datetime.datetime(2018, 12, 15)
end_date = datetime.datetime(2024, 1, 30)

# Generate images every 30 days
current_date = start_date
while current_date <= end_date:
    # Define the date range for the current image
    date_range_start = current_date.strftime('%Y-%m-%d')
    next_date = current_date + datetime.timedelta(days=30)
    date_range_end = next_date.strftime('%Y-%m-%d')

    # Get the image collection for the current date range
    sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterDate(date_range_start, date_range_end) \
        .filterBounds(aoi) \
        .map(maskClouds) \
        .median() 
        

    # Clip the image to the AOI
    clippedImage = sentinelCollection.clip(aoi)

    # Cast all bands of the clipped image to Float32
    castedImage = castImage(clippedImage)

    # Export the casted image
    exportImage(castedImage, current_date)

    # Update the current date
    current_date = next_date

print("Images export tasks started.")

Images export tasks started.


## A new aproach

In [7]:
!pip install geopy


  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\owd1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import ee
from geopy.geocoders import Nominatim

In [10]:
import ee
ee.Initialize()


In [11]:
import ee
from geopy.geocoders import Nominatim

# Initialize Earth Engine library
ee.Initialize()

# Get user input for the location
place_name = input("Enter a city, state, or country: ")

# Try to get the place from the country dataset
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
country = countries.filter(ee.Filter.eq('ADM0_NAME', place_name))

# Check if the country collection is not empty
if country.size().getInfo() > 0:
    aoi = country.geometry()
    print(f"Using country geometry for {place_name}")
else:
    # Try to get the place from the state dataset
    states = ee.FeatureCollection("FAO/GAUL/2015/level1")
    state = states.filter(ee.Filter.eq('ADM1_NAME', place_name))
    if state.size().getInfo() > 0:
        aoi = state.geometry()
        print(f"Using state geometry for {place_name}")
    else:
        # Assume it's a city and use geocoding to get the location
        geolocator = Nominatim(user_agent="my_app")
        location = geolocator.geocode(place_name)

        if location is None:
            print("Could not geocode the place name.")
            exit()
        else:
            print(f"Location: {location.address}")
            print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")

            # Create an ee.Geometry.Point
            point = ee.Geometry.Point([location.longitude, location.latitude])

            # Define the AOI by buffering the point (e.g., 10 km radius)
            aoi = point.buffer(10000)  # Buffer in meters
            print(f"Using buffered point geometry for {place_name}")

# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2024-03-01', '2024-03-28') \
    .filterBounds(aoi)

# Compute the median image from the collection
median_image = sentinelCollection.median()

# Clip the median image to the AOI
clippedImage = median_image.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)

# Export the casted image, specifying scale and region
task = ee.batch.Export.image.toDrive(**{
    'image': castedImage,
    'description': f'{place_name}_Forest',
    'scale': 10,
    'region': aoi.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})
task.start()

print("Images export tasks started.")


Location: الرياض, محافظة الرياض, منطقة الرياض, 11131, السعودية
Latitude: 24.638916, Longitude: 46.7160104
Using buffered point geometry for Riyadh
Images export tasks started.


In [13]:
import ee
from geopy.geocoders import Nominatim

# Initialize Earth Engine library
ee.Initialize()

# Get user input for the location
place_name = input("Enter a city, state, or country: ")

# Try to get the place from the country dataset
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
country = countries.filter(ee.Filter.eq('ADM0_NAME', place_name))

# Check if the country collection is not empty
if country.size().getInfo() > 0:
    aoi = country.geometry()
    print(f"Using country geometry for {place_name}")
else:
    # Try to get the place from the state dataset
    states = ee.FeatureCollection("FAO/GAUL/2015/level1")
    state = states.filter(ee.Filter.eq('ADM1_NAME', place_name))
    if state.size().getInfo() > 0:
        aoi = state.geometry()
        print(f"Using state geometry for {place_name}")
    else:
        # Assume it's a city and use geocoding to get the location
        geolocator = Nominatim(user_agent="my_app")
        location = geolocator.geocode(place_name)

        if location is None:
            print("Could not geocode the place name.")
            exit()
        else:
            print(f"Location: {location.address}")
            print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")

            # Create an ee.Geometry.Point
            point = ee.Geometry.Point([location.longitude, location.latitude])

            # Define the AOI by buffering the point (e.g., 10 km radius)
            aoi = point.buffer(10000)  # Buffer in meters
            print(f"Using buffered point geometry for {place_name}")

# Define the bands to select
bands = ['B8', 'B4', 'B3', 'B2']

# Corrected create_grid function
def create_grid(aoi, scale):
    """Create a grid over the AOI with the specified scale (in meters)."""
    # Get the bounding box coordinates
    bounds = aoi.bounds()
    coords = ee.List(bounds.coordinates().get(0))

    # Extract the x and y coordinates from the bounding box
    xs = coords.map(lambda point: ee.Number(ee.List(point).get(0)))
    ys = coords.map(lambda point: ee.Number(ee.List(point).get(1)))

    xmin = xs.reduce(ee.Reducer.min())
    xmax = xs.reduce(ee.Reducer.max())
    ymin = ys.reduce(ee.Reducer.min())
    ymax = ys.reduce(ee.Reducer.max())

    # Convert scale from meters to degrees (approximate)
    scale_deg = ee.Number(scale).divide(111319.9)

    # Generate lists of x and y coordinates
    x_coords = ee.List.sequence(xmin, xmax, scale_deg)
    y_coords = ee.List.sequence(ymin, ymax, scale_deg)

    # Create grid cells
    def make_cells(x):
        x = ee.Number(x)
        def make_cell(y):
            y = ee.Number(y)
            x1 = x
            y1 = y
            x2 = x.add(scale_deg)
            y2 = y.add(scale_deg)
            cell_geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            return ee.Feature(cell_geom)
        return y_coords.map(make_cell)

    cells = x_coords.map(make_cells).flatten()
    grid = ee.FeatureCollection(cells)
    return grid.filterBounds(aoi)

scale = 1000  # 1 km grid cells
grid = create_grid(aoi, scale)

# Get the list of cells
cells_list = grid.toList(grid.size())
num_cells = cells_list.size().getInfo()

print(f"Processing {num_cells} cells")

# Iterate over each cell
for i in range(num_cells):
    cell = ee.Feature(cells_list.get(i))
    cell_geometry = cell.geometry()

    # Filter the Sentinel-2 ImageCollection for images within the cell and time frame
    sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterDate('2023-03-01', '2023-03-28') \
        .filterBounds(cell_geometry) \
        .select(bands + ['QA60'])

    # Function to calculate cloud cover over the cell
    def calculate_cloud_cover(image):
        # Create a cloud mask using the 'QA60' band
        qa = image.select('QA60')
        # Bits 10 and 11 are clouds and cirrus
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        # Both flags set to zero indicates clear conditions
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
            qa.bitwiseAnd(cirrusBitMask).eq(0))
        # Calculate the cloud percentage over the region
        totalPixels = image.select(0).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=cell_geometry,
            scale=10,
            maxPixels=1e9
        ).values().get(0)
        cloudPixels = image.select(0).updateMask(mask.Not()).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=cell_geometry,
            scale=10,
            maxPixels=1e9
        ).values().get(0)
        cloudPercentage = ee.Number(cloudPixels).divide(ee.Number(totalPixels)).multiply(100)
        # Add the cloud percentage as a property
        return image.set('CLOUD_COVERAGE_CELL', cloudPercentage)

    # Map the function over the images
    sentinelCollection = sentinelCollection.map(calculate_cloud_cover)
    # Filter images with cloud cover less than threshold
    cloud_threshold = 5
    sentinelCollection = sentinelCollection.filter(ee.Filter.lt('CLOUD_COVERAGE_CELL', cloud_threshold))
    # Select the image with least cloud cover
    best_image = sentinelCollection.sort('CLOUD_COVERAGE_CELL').first()

    if best_image:
        # Extract the four bands
        best_image = best_image.select(bands)
        # Clip to the cell geometry
        clipped_image = best_image.clip(cell_geometry)
        # Cast to float32
        casted_image = clipped_image.toFloat()

        # Export the image to Drive
        task = ee.batch.Export.image.toDrive(**{
            'image': casted_image,
            'description': f'{place_name}_cell_{i}',
            'folder': f'{place_name}_folder',
            'scale': 10,
            'region': cell_geometry,
            'fileFormat': 'GeoTIFF',
            'maxPixels': 1e9
        })
        task.start()
        print(f"Export task started for cell {i}")
    else:
        print(f"No suitable image found for cell {i}")

print("Image export tasks started.")


Location: الرياض, محافظة الرياض, منطقة الرياض, 11131, السعودية
Latitude: 24.638916, Longitude: 46.7160104
Using buffered point geometry for Riyadh
Processing 376 cells
Export task started for cell 0
Export task started for cell 1
Export task started for cell 2
Export task started for cell 3
Export task started for cell 4
Export task started for cell 5
Export task started for cell 6
Export task started for cell 7
Export task started for cell 8
Export task started for cell 9
Export task started for cell 10
Export task started for cell 11
Export task started for cell 12
Export task started for cell 13
Export task started for cell 14
Export task started for cell 15
Export task started for cell 16
Export task started for cell 17
Export task started for cell 18
Export task started for cell 19
Export task started for cell 20
Export task started for cell 21
Export task started for cell 22
Export task started for cell 23
Export task started for cell 24
Export task started for cell 25
Export tas